## Dependences

In [1]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes>=0.39.0
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 47.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 8.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uaf34nd3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uaf34nd3
  Resolved https://github.com/huggingface/transformers.git to commit 30ed3adf474aaf2972ab56f5624089bc24a6adf3
  Installing bu

In [1]:
!nvidia-smi

Thu Jul 13 13:43:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070         Off| 00000000:01:00.0 Off |                  N/A |
|  0%   55C    P8               16W / 220W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!free

              total        used        free      shared  buff/cache   available
Mem:      181108932    10473132    73337164     5609268    97298636   163382608
Swap:             0           0           0


## Imports

In [3]:
import os
import torch
import time
import datetime
import sys
import traceback
import gc

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## Loading the model

In [2]:
model_name = 'legendhasit/xgen-7b-8k-inst-8bit'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//10.43.0.1'), PosixPath('443'), PosixPath('tcp')}
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent:

## Inference

In [4]:
# Parameters:
# - DATASET_NAME
# - HEADER
# - PROMPT_TEMPLATE
# - samples in dataset 'input/<dataset_name>'
# - instructions in 'instructions.txt'

def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

mkdir('input')

# Define prompt template
# ==========================================================================================
HEADER = (
    "A chat between a curious human and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the human's questions."
)

PROMPT_TEMPLATE = HEADER + """

### Human: {instruction}

{article}

### Assistant:"""

# Load samples from dataset
# ==========================================================================================
DATASET_NAME = 'fredsum'
samples = os.listdir('input/' + DATASET_NAME + '/texts')
n_samples = len(samples)
print('Found', n_samples, 'samples')

# Load instructions
# ==========================================================================================
instruction_file = open('instructions.txt', 'r', encoding='utf-8')
instructions = instruction_file.readlines()
instruction_file.close()
n_instructions = len(instructions)
for i in range(n_instructions):
    instructions[i] = instructions[i].replace('\n', '')
print('Found', n_instructions, 'instructions')

Found 10 samples
Found 6 instructions


In [7]:
# Inference
# ==========================================================================================
initial_time = time.time()
skipped_samples = 0

mkdir('intermediate')
mkdir('intermediate/' + DATASET_NAME)

print('Starting computation...')

# For each instruction
for instruction_n in range(n_instructions):

    # Read instruction and create prompt
    instruction = instructions[instruction_n]
    
    # For each sample in dataset
    for sample_n in range(n_samples):

        # Estimate completion and time.
        cur_samples = instruction_n * n_samples + sample_n - skipped_samples
        tot_samples = n_instructions * n_samples - skipped_samples
        progress = cur_samples / tot_samples
        pct = round(progress * 100, 1)
        print('Prompting instruction N' + str(instruction_n + 1) + '/' + str(n_instructions) + ' on sample N' + str(sample_n + 1) + '/' + str(n_samples))
        print('-- Completion: ' + str(pct) + '%')
        if cur_samples > 0:
            approx_total = (time.time() - initial_time) / cur_samples * tot_samples
            approx_remaining = approx_total * (1 - progress)
            print('-- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')
        
        # Read sample and generate prompt
        sample_file_path = 'input/' + DATASET_NAME + '/texts/' + samples[sample_n]
        sample_file = open(sample_file_path, 'r', encoding='utf-8')
        sample = sample_file.read()
        sample_file.close()
        prompt = PROMPT_TEMPLATE.format(instruction=instruction, article=sample)
        
        # Find target file
        target_file_path = 'intermediate/' + DATASET_NAME + '/' + str(instruction_n + 1) + '_' + str(sample_n + 1) + '.txt'
        if os.path.isfile(target_file_path):
            print('-- Found intermediate result file \'' + target_file_path + '\', skipped.')
            skipped_samples += 1
            continue
            
        try:
        
            # Sample one answer
            input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
            sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
            output = tokenizer.decode(sample[0]).strip()

            # Save answer in file
            target_file = open(target_file_path, 'w', encoding='utf-8')
            target_file.write(output)
            target_file.close()

            del input_ids
            del sample
        
        except:
            print('Could not compute prompt:')
            print(prompt)
            traceback.print_exc()

        gc.collect()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

Starting computation...
Prompting instruction N1/6 on sample N1/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_1.txt', skipped.
Prompting instruction N1/6 on sample N2/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_2.txt', skipped.
Prompting instruction N1/6 on sample N3/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_3.txt', skipped.
Prompting instruction N1/6 on sample N4/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_4.txt', skipped.
Prompting instruction N1/6 on sample N5/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_5.txt', skipped.
Prompting instruction N1/6 on sample N6/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_6.txt', skipped.
Prompting instruction N1/6 on sample N7/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_7.txt', skipped.
Prompting instruction N1/6 on sample N8/10
-- Completion: 0.0%
-- 

In [9]:
del input_ids
del sample
del output

## Calculs de scores

In [33]:
!pip install rouge_score rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu
!pip install nltk

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=7afcf4fd12bb20d962442dc2662012abd1ec10704de49142044f1fad7b14b3dc
  Stored in directory: c:\users\jules\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import evaluate

c:\Users\Jules\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Method and variables
# ==========================================================================================
print('Starting scores computation...')
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

SUMMARIES_PER_SAMPLE = 4
INTERMEDIATE_SUBPATH = 'fr/'
STORAGE_FILE_NAME = 'scores_fr'
PREPROCESS_SUMMARIES = False

# Script itself
# ==========================================================================================

# Find output file for CSV scores
mkdir('output')
mkdir('output/' + DATASET_NAME)
storage_file = open('output/' + DATASET_NAME + '/' + STORAGE_FILE_NAME + '.csv', 'w', encoding='utf-8')
storage_file.write('path;rouge2;rougel;bleu;bertscore\n')

target_file_paths = []
references = []
predictions = []

# For each instruction
for instruction_n in range(n_instructions):

    # Read instruction and create prompt
    instruction = instructions[instruction_n]
    
    # For each sample in dataset
    for sample_n in range(n_samples):
        
        # Find target file
        target_file_path = 'intermediate/' + DATASET_NAME + '/' + INTERMEDIATE_SUBPATH + str(instruction_n + 1) + '_' + str(sample_n + 1) + '.txt'
        if not os.path.isfile(target_file_path): # A MODIFIER : SI UN RESUME N'A PAS ETE GENERE
            print('-- Found no intermediate result file \'' + target_file_path + '\', skipped.')
            continue
        
        # Read sample and generate prompt -> Keep summary
        sub_references = []
        for reference in range(SUMMARIES_PER_SAMPLE):
            summary_file_path = 'input/' + DATASET_NAME + '/summaries/sample_' + str(sample_n + 1) + '_' + str(reference + 1) + '.txt'
            summary_file = open(summary_file_path, 'r', encoding='utf-8')
            sub_references.append(summary_file.read())
            summary_file.close()
        references.append(sub_references)

        # Access generated summary
        target_file = open(target_file_path, 'r', encoding='utf-8')
        prediction = target_file.read()
        target_file.close()

        # Process answer
        if PREPROCESS_SUMMARIES:
            separator = "### Assistant:"
            prediction = prediction[prediction.index(separator) + len(separator):]
            if prediction[0] == " ": # Enlever l'espace devant
                prediction = prediction[1:]
            prediction = prediction[:-len("<|endoftext|>") - 2]
            #print(prediction)
            #print('---------------------------------------')
        
        # Add prediction
        target_file_paths.append(target_file_path)
        predictions.append(prediction)

# Calculate metrics
result_rouge = rouge.compute(predictions=predictions, references=references)
result_bleu = bleu.compute(predictions=predictions, references=references)
result_bertscore = bertscore.compute(predictions=predictions, references=references, lang='fr', rescale_with_baseline=True, verbose=True)

print(result_rouge)
print(result_bleu)
print(result_bertscore)

if False:

    # Write to csv
    # Format: PATH | ROUGE2 | ROUGEL | BLEU | BERTScore
    for i in range(len(target_file_paths)):
        ligne = target_file_path[i] + ';'
        ligne += ';' + str(result_rouge['rouge2']) + ";" + str(result_rouge['rougeL'])
        ligne += ";" + str(result_bleu['bleu'])
        ligne += ";" + str(result_bertscore['f1'][0])
        
        storage_file.write(ligne + '\n')

    storage_file.close()
    print('Done!')

Starting scores computation...
-- Found no intermediate result file 'intermediate/fredsum/fr/1_10.txt', skipped.
-- Found no intermediate result file 'intermediate/fredsum/fr/2_10.txt', skipped.
-- Found no intermediate result file 'intermediate/fredsum/fr/3_10.txt', skipped.
-- Found no intermediate result file 'intermediate/fredsum/fr/4_10.txt', skipped.
-- Found no intermediate result file 'intermediate/fredsum/fr/5_10.txt', skipped.
-- Found no intermediate result file 'intermediate/fredsum/fr/6_10.txt', skipped.
calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:42<?, ?it/s]


KeyboardInterrupt: 